In [295]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 
torch.cuda.is_available()

True

In [296]:
data_name = "gsm8k"

if data_name == "gsm8k":
    path = '/root/COLM/dataset/gsm8k_unified_data_v4.csv'
    data_all = pd.read_csv(path, encoding="utf-8")

print(data_all.shape, data_name)
merged_df.head()

(8792, 37) gsm8k


,question,gold_answer,split,subject_x,sr_gemma-7b-it_CHAT,sr_llama2-13b-chat_CHAT,sr_mistral-7b-inst_CHAT,maj_gemma-7b-it_CHAT,maj_llama2-13b-chat_CHAT,maj_mistral-7b-inst_CHAT,...,auto_answers_gemma-7b,auto_answers_llama2-7b-lm,auto_answers_metamath-7b,auto_answers_mistral-7b-lm,auto_answers_olmo-7b,auto_answers_opt-6.7b,rl_gemma-7b,rl_mistral-7b-lm,rl_llama2-13b-chat,rl_gemma-7b-it
0,For which of these two scenarios does the main...,D,train,moral_scenarios,0.0,0.8,0.2,0.0,1.0,0.0,...,"['B', 'D', 'D', 'C', 'C', 'D', 'D', 'C', 'D', ...","['A', 'B', 'B', 'D', 'D', 'D', 'D', 'D', 'D', ...","['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', ...","['D', 'D', 'D', 'D', 'D', 'D', 'D', 'B', 'D', ...","['C', 'C', 'D', 'D', 'D', 'D', 'D', 'D', 'D', ...","['C', 'C', 'D', 'C', 'B', 'D', 'D', 'D', 'D', ...",0.238095,0.238095,0.238095,0.000000
1,For which of these two scenarios does the main...,A,train,moral_scenarios,0.1,0.0,0.5,0.0,0.0,1.0,...,"['B', 'A', 'B', 'B', 'B', 'A', 'B', 'A', 'B', ...","['B', 'B', 'A', 'B', 'A', 'B', 'A', 'A', 'B', ...","['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', ...","['A', 'A', 'A', 'B', 'A', 'A', 'B', 'B', 'A', ...","['D', 'C', 'B', 'B', 'B', 'A', 'D', 'D', 'B', ...","['B', 'A', 'C', 'B', 'B', 'C', 'C', 'D', 'B', ...",0.142857,1.000000,0.142857,0.142857
2,For which of these two scenarios does the main...,D,train,moral_scenarios,0.1,0.9,0.4,0.0,1.0,1.0,...,"['B', 'C', 'D', 'D', 'D', 'B', 'D', 'B', 'A', ...","['D', 'D', 'D', 'INVALID', 'A', 'D', 'C', 'D',...","['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', ...","['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', ...","['B', 'D', 'D', 'D', 'A', 'B', 'C', 'INVALID',...","['C', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'B', ...",0.238095,0.238095,0.238095,0.000000
3,For which of these two scenarios does the main...,B,train,moral_scenarios,0.4,0.0,0.6,0.0,0.0,1.0,...,"['A', 'B', 'B', 'A', 'B', 'A', 'A', 'B', 'B', ...","['A', 'A', 'D', 'B', 'B', 'D', 'A', 'A', 'A', ...","['D', 'D', 'D', 'D', 'D', 'D', 'D', 'B', 'D', ...","['A', 'B', 'A', 'D', 'A', 'A', 'A', 'A', 'B', ...","['D', 'A', 'C', 'D', 'C', 'D', 'A', 'C', 'D', ...","['B', 'INVALID', 'A', 'D', 'D', 'D', 'B', 'D',...",0.428571,0.428571,0.428571,0.428571
4,For which of these two scenarios does the main...,D,train,moral_scenarios,0.0,0.6,0.7,0.0,1.0,1.0,...,"['D', 'C', 'D', 'D', 'D', 'D', 'D', 'D', 'D', ...","['D', 'D', 'D', 'INVALID', 'D', 'D', 'D', 'D',...","['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', ...","['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', ...","['D', 'D', 'D', 'A', 'D', 'B', 'C', 'D', 'B', ...","['D', 'D', 'C', 'D', 'C', 'D', 'D', 'B', 'D', ...",0.238095,0.238095,0.238095,0.000000


In [298]:
data_all.rename(columns = {'maj_gemma-7b':'maj_gemma-7b_LM', 'maj_metamath-7b':'maj_metamath-7b_LM', 'maj_mistral-7b-lm':'maj_mistral-7b-lm_LM', "maj_llama2-7b-lm":"maj_llama2-7b-lm_LM",\
                              'maj_gemma-7b-it':'maj_gemma-7b-it_CHAT', 'maj_llama2-13b-chat':'maj_llama2-13b-chat_CHAT', 'maj_mistral-7b-inst':'maj_mistral-7b-inst_CHAT', \
                              "auto_answers_gemma-7b":"auto_answers_maj_gemma-7b_LM", "auto_answers_metamath-7b":"auto_answers_maj_metamath-7b_LM", "auto_answers_mistral-7b-lm":"auto_answers_maj_mistral-7b-lm_LM", "auto_answers_llama2-7b-lm":"auto_answers_maj_llama2-7b-lm_LM",\
                             'auto_answers_gemma-7b-it':'auto_answers_maj_gemma-7b-it_CHAT', 'auto_answers_llama2-13b-chat':'auto_answers_maj_llama2-13b-chat_CHAT', 'auto_answers_mistral-7b-inst':'auto_answers_maj_mistral-7b-inst_CHAT'}, inplace = True)\

column_names = data_all.columns.tolist()
column_names

['question',
 'gold_answer',
 'split',
 'sr_falcon-7b',
 'sr_gemma-7b',
 'sr_llama2-7b-lm',
 'sr_metamath-7b',
 'sr_mistral-7b-lm',
 'sr_olmo-7b',
 'sr_opt-6.7b',
 'maj_falcon-7b',
 'maj_gemma-7b_LM',
 'maj_llama2-7b-lm_LM',
 'maj_metamath-7b_LM',
 'maj_mistral-7b-lm_LM',
 'maj_olmo-7b',
 'maj_opt-6.7b',
 'auto_answers_falcon-7b',
 'auto_answers_maj_gemma-7b_LM',
 'auto_answers_maj_llama2-7b-lm_LM',
 'auto_answers_maj_metamath-7b_LM',
 'auto_answers_maj_mistral-7b-lm_LM',
 'auto_answers_olmo-7b',
 'auto_answers_opt-6.7b',
 'sr_gemma-7b-it',
 'sr_llama2-13b-chat',
 'sr_mistral-7b-inst',
 'maj_gemma-7b-it_CHAT',
 'maj_llama2-13b-chat_CHAT',
 'maj_mistral-7b-inst_CHAT',
 'auto_answers_maj_gemma-7b-it_CHAT',
 'auto_answers_maj_llama2-13b-chat_CHAT',
 'auto_answers_maj_mistral-7b-inst_CHAT',
 'rl_gemma-7b',
 'rl_mistral-7b-lm',
 'rl_llama2-13b-chat',
 'rl_gemma-7b-it']

In [299]:
#retain required colmns
if data_name == 'gsm8k':
    data_all = data_all[['question', 'gold_answer', 'split', 'maj_gemma-7b_LM', 'maj_metamath-7b_LM', 'maj_mistral-7b-lm_LM', 'maj_gemma-7b-it_CHAT', 
                     'maj_llama2-13b-chat_CHAT', 'maj_mistral-7b-inst_CHAT', "auto_answers_maj_gemma-7b_LM", "auto_answers_maj_metamath-7b_LM", \
                     "auto_answers_maj_mistral-7b-lm_LM", 'auto_answers_maj_gemma-7b-it_CHAT','auto_answers_maj_llama2-13b-chat_CHAT', \
                     'auto_answers_maj_mistral-7b-inst_CHAT']]
elif data_name == 'mmlu':
    data_all = data_all[['question', 'gold_answer', 'split', "subject", 'maj_gemma-7b_LM', 'maj_metamath-7b_LM', 'maj_mistral-7b-lm_LM', 'maj_gemma-7b-it_CHAT', 'maj_llama2-7b-lm_LM',
                     'maj_llama2-13b-chat_CHAT', 'maj_mistral-7b-inst_CHAT', "auto_answers_maj_gemma-7b_LM", "auto_answers_maj_metamath-7b_LM", "auto_answers_maj_llama2-7b-lm_LM", \
                     "auto_answers_maj_mistral-7b-lm_LM", 'auto_answers_maj_gemma-7b-it_CHAT','auto_answers_maj_llama2-13b-chat_CHAT', \
                     'auto_answers_maj_mistral-7b-inst_CHAT']]
data_all.head()

,question,gold_answer,split,maj_gemma-7b_LM,maj_metamath-7b_LM,maj_mistral-7b-lm_LM,maj_gemma-7b-it_CHAT,maj_llama2-13b-chat_CHAT,maj_mistral-7b-inst_CHAT,auto_answers_maj_gemma-7b_LM,auto_answers_maj_metamath-7b_LM,auto_answers_maj_mistral-7b-lm_LM,auto_answers_maj_gemma-7b-it_CHAT,auto_answers_maj_llama2-13b-chat_CHAT,auto_answers_maj_mistral-7b-inst_CHAT
0,Natalia sold clips to 48 of her friends in Apr...,72.0,train,1.0,1.0,1.0,1.0,1.0,1.0,"[96.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72....","[72.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72....","[72.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72....","[72.0, 24.0, 72.0, 72.0, 72.0, 'INVALID', 72.0...","[72.0, 72.0, 72.0, 72.0, 720.0, 72.0, 24.0, 72...","[96.0, 24.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72...."
1,Weng earns $12 an hour for babysitting. Yester...,10.0,train,1.0,1.0,1.0,0.0,0.0,1.0,"[10.0, 10.0, 600.0, 10.0, 10.0, 10000000000.0,...","[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10....","[10.0, 99.0, 10.0, 5.0, 10.0, 60.0, 10.0, 60.0...","[6.0, 6.0, 6.0, 6.0, 6.0, 'INVALID', 'INVALID'...","[60.0, 9.96, 60.0, 60.0, 60.0, 60.0, 'INVALID'...","[9.99992, 10.0, 12.0, 10.4, 10.0, 9.996, 12.0,..."
2,Betty is saving money for a new wallet which c...,5.0,train,1.0,1.0,0.0,0.0,1.0,1.0,"[5.0, 45.0, 55.0, 5.0, 10.0, 5.0, 35.0, 5.0, 5...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[55.0, 55.0, 155.0, 55.0, 5.0, 55.0, 55.0, 45....","[-5.0, 30.0, 'INVALID', 45.0, 30.0, 35.0, 35.0...","[45.0, 45.0, 45.0, 5.0, 55.0, 5.0, 5.0, 55.0, ...","[5.0, 75.0, 5.0, 105.0, 55.0, 5.0, 92.5, 25.0,..."
3,"Julie is reading a 120-page book. Yesterday, s...",42.0,train,1.0,1.0,0.0,1.0,0.0,1.0,"[84.0, 42.0, 84.0, 42.0, 42.0, 42.0, 84.0, 54....","[42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42....","[96.0, 84.0, 96.0, 96.0, 24.0, 48.0, 25.0, 72....","[72.0, 84.0, 72.0, 42.0, 48.0, 84.0, 24.0, 42....","[60.0, 64.0, 60.0, 44.0, 96.0, 84.0, 42.0, 96....","[42.0, 72.0, 42.0, 54.0, 12.0, 60.0, 24.0, 'IN..."
4,James writes a 3-page letter to 2 different fr...,624.0,train,0.0,1.0,0.0,0.0,0.0,0.0,"[312.0, 144.0, 600.0, 624.0, 468.0, 104.0, 312...","[624.0, 624.0, 624.0, 624.0, 624.0, 624.0, 624...","[144.0, 208.0, 624.0, 312.0, 312.0, 312.0, 312...","[208.0, 3.0, 104.0, 104.0, 'INVALID', 156.0, 1...","[312.0, 312.0, 312.0, 312.0, 624.0, 312.0, 208...","[624.0, 312.0, 1296.0, 312.0, 312.0, 312.0, 62..."


In [300]:
def mostfreqentval(mylist, gold_val):
    if data_name == "gsm8k":
        my_list = [ float(item) for item in mylist if item != "INVALID"]
        counter = Counter(my_list)
        most_common_values = counter.most_common()
        if len(most_common_values) == 0:
            topfrq = 0.9999989898999998989899
        else:
            topfrq = most_common_values[0][0]
        if float(topfrq) == float(gold_val):
            return 1
        else:
            return 0
    elif data_name == "mmlu":
        my_list = [ item.strip() for item in mylist]
        counter = Counter(my_list)
        most_common_values = counter.most_common()
        if len(most_common_values) == 0:
            topfrq = 0.9999989898999998989899
        else:
            topfrq = most_common_values[0][0]
        if topfrq.strip() == gold_val.strip():
            return 1
        else:
            return 0

if data_name == "gsm8k":
    kepping_llms = ["maj_gemma-7b-it_CHAT", "maj_llama2-13b-chat_CHAT", "maj_mistral-7b-inst_CHAT", "maj_mistral-7b-lm_LM", "maj_gemma-7b_LM", "maj_metamath-7b_LM"]
elif data_name == "mmlu":
    kepping_llms = ["maj_gemma-7b-it_CHAT", "maj_llama2-13b-chat_CHAT", "maj_mistral-7b-inst_CHAT", "maj_mistral-7b-lm_LM", "maj_gemma-7b_LM", "maj_metamath-7b_LM", "maj_llama2-7b-lm_LM"]
 
for llm_name in kepping_llms:
    del data_all[llm_name]
    data_all[llm_name] = [ mostfreqentval(eval(item), gval) for item, gval in zip(data_all["auto_answers_"+str(llm_name)].values.tolist(), data_all["gold_answer"].values.tolist())]

data_all.head()

,question,gold_answer,split,auto_answers_maj_gemma-7b_LM,auto_answers_maj_metamath-7b_LM,auto_answers_maj_mistral-7b-lm_LM,auto_answers_maj_gemma-7b-it_CHAT,auto_answers_maj_llama2-13b-chat_CHAT,auto_answers_maj_mistral-7b-inst_CHAT,maj_gemma-7b-it_CHAT,maj_llama2-13b-chat_CHAT,maj_mistral-7b-inst_CHAT,maj_mistral-7b-lm_LM,maj_gemma-7b_LM,maj_metamath-7b_LM
0,Natalia sold clips to 48 of her friends in Apr...,72.0,train,"[96.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72....","[72.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72....","[72.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72....","[72.0, 24.0, 72.0, 72.0, 72.0, 'INVALID', 72.0...","[72.0, 72.0, 72.0, 72.0, 720.0, 72.0, 24.0, 72...","[96.0, 24.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72....",1,1,1,1,1,1
1,Weng earns $12 an hour for babysitting. Yester...,10.0,train,"[10.0, 10.0, 600.0, 10.0, 10.0, 10000000000.0,...","[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10....","[10.0, 99.0, 10.0, 5.0, 10.0, 60.0, 10.0, 60.0...","[6.0, 6.0, 6.0, 6.0, 6.0, 'INVALID', 'INVALID'...","[60.0, 9.96, 60.0, 60.0, 60.0, 60.0, 'INVALID'...","[9.99992, 10.0, 12.0, 10.4, 10.0, 9.996, 12.0,...",0,0,1,1,1,1
2,Betty is saving money for a new wallet which c...,5.0,train,"[5.0, 45.0, 55.0, 5.0, 10.0, 5.0, 35.0, 5.0, 5...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[55.0, 55.0, 155.0, 55.0, 5.0, 55.0, 55.0, 45....","[-5.0, 30.0, 'INVALID', 45.0, 30.0, 35.0, 35.0...","[45.0, 45.0, 45.0, 5.0, 55.0, 5.0, 5.0, 55.0, ...","[5.0, 75.0, 5.0, 105.0, 55.0, 5.0, 92.5, 25.0,...",0,1,1,0,1,1
3,"Julie is reading a 120-page book. Yesterday, s...",42.0,train,"[84.0, 42.0, 84.0, 42.0, 42.0, 42.0, 84.0, 54....","[42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42....","[96.0, 84.0, 96.0, 96.0, 24.0, 48.0, 25.0, 72....","[72.0, 84.0, 72.0, 42.0, 48.0, 84.0, 24.0, 42....","[60.0, 64.0, 60.0, 44.0, 96.0, 84.0, 42.0, 96....","[42.0, 72.0, 42.0, 54.0, 12.0, 60.0, 24.0, 'IN...",1,0,1,0,1,1
4,James writes a 3-page letter to 2 different fr...,624.0,train,"[312.0, 144.0, 600.0, 624.0, 468.0, 104.0, 312...","[624.0, 624.0, 624.0, 624.0, 624.0, 624.0, 624...","[144.0, 208.0, 624.0, 312.0, 312.0, 312.0, 312...","[208.0, 3.0, 104.0, 104.0, 'INVALID', 156.0, 1...","[312.0, 312.0, 312.0, 312.0, 624.0, 312.0, 208...","[624.0, 312.0, 1296.0, 312.0, 312.0, 312.0, 62...",0,0,0,0,0,1


In [301]:
if data_name == 'gsm8k':
    keep_llms_old = ["maj_gemma-7b-it_CHAT", "maj_llama2-13b-chat_CHAT", "maj_mistral-7b-inst_CHAT", "maj_mistral-7b-lm_LM", "maj_gemma-7b_LM", "maj_metamath-7b_LM"]
elif data_name == 'mmlu':
    keep_llms_old = ["maj_metamath-7b_LM", "maj_gemma-7b-it_CHAT", "maj_mistral-7b-inst_CHAT", "maj_llama2-7b-lm_LM", "maj_llama2-13b-chat_CHAT", "maj_mistral-7b-lm_LM", "maj_gemma-7b_LM"]

print(keep_llms_old)
keep_llms = keep_llms_old[::-1]
print(keep_llms)
    
for col_name in keep_llms:
    data_all[col_name] = [ int(item) for item in data_all[col_name].values.tolist()]

for i in range(len(keep_llms)-1):
    subset_llms =  keep_llms[0:i+2]
    id2mod = { i:val for i, val in enumerate(subset_llms)} 
    mod2id = { val:1 for i, val in enumerate(subset_llms)}
    print(subset_llms)
    data_all[f'maj_{i+2}'] = [ [ int(val) for val in item ] for item in data_all[subset_llms].values.tolist()]
    data_all[f'maj_{i+2}_model'] = [ [ id2mod[i] for i, val in enumerate(item) if id2mod[i] in subset_llms and val != 0] for item in data_all[f'maj_{i+2}'].tolist()]

data.head()

['maj_gemma-7b-it_CHAT', 'maj_llama2-13b-chat_CHAT', 'maj_mistral-7b-inst_CHAT', 'maj_mistral-7b-lm_LM', 'maj_gemma-7b_LM', 'maj_metamath-7b_LM']
['maj_metamath-7b_LM', 'maj_gemma-7b_LM', 'maj_mistral-7b-lm_LM', 'maj_mistral-7b-inst_CHAT', 'maj_llama2-13b-chat_CHAT', 'maj_gemma-7b-it_CHAT']
['maj_metamath-7b_LM', 'maj_gemma-7b_LM']
['maj_metamath-7b_LM', 'maj_gemma-7b_LM', 'maj_mistral-7b-lm_LM']
['maj_metamath-7b_LM', 'maj_gemma-7b_LM', 'maj_mistral-7b-lm_LM', 'maj_mistral-7b-inst_CHAT']
['maj_metamath-7b_LM', 'maj_gemma-7b_LM', 'maj_mistral-7b-lm_LM', 'maj_mistral-7b-inst_CHAT', 'maj_llama2-13b-chat_CHAT']
['maj_metamath-7b_LM', 'maj_gemma-7b_LM', 'maj_mistral-7b-lm_LM', 'maj_mistral-7b-inst_CHAT', 'maj_llama2-13b-chat_CHAT', 'maj_gemma-7b-it_CHAT']


,question,gold_answer,split,maj_gemma-7b_LM,maj_metamath-7b_LM,maj_mistral-7b-lm_LM,maj_gemma-7b-it_CHAT,maj_llama2-13b-chat_CHAT,maj_mistral-7b-inst_CHAT,auto_answers_maj_gemma-7b_LM,...,maj_2,maj_2_model,maj_3,maj_3_model,maj_4,maj_4_model,maj_5,maj_5_model,maj_6,maj_6_model
0,Natalia sold clips to 48 of her friends in Apr...,72.0,train,1,1,1,1,1,1,"[96.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72.0, 72....",...,"[1, 1]","[maj_gemma-7b-it_CHAT, maj_llama2-13b-chat_CHAT]","[1, 1, 1]","[maj_gemma-7b-it_CHAT, maj_llama2-13b-chat_CHA...","[1, 1, 1, 1]","[maj_gemma-7b-it_CHAT, maj_llama2-13b-chat_CHA...","[1, 1, 1, 1, 1]","[maj_gemma-7b-it_CHAT, maj_llama2-13b-chat_CHA...","[1, 1, 1, 1, 1, 1]","[maj_gemma-7b-it_CHAT, maj_llama2-13b-chat_CHA..."
1,Weng earns $12 an hour for babysitting. Yester...,10.0,train,1,1,1,0,0,1,"[10.0, 10.0, 600.0, 10.0, 10.0, 10000000000.0,...",...,"[0, 0]",[],"[0, 0, 1]",[maj_mistral-7b-inst_CHAT],"[0, 0, 1, 1]","[maj_mistral-7b-inst_CHAT, maj_mistral-7b-lm_LM]","[0, 0, 1, 1, 1]","[maj_mistral-7b-inst_CHAT, maj_mistral-7b-lm_L...","[0, 0, 1, 1, 1, 1]","[maj_mistral-7b-inst_CHAT, maj_mistral-7b-lm_L..."
2,Betty is saving money for a new wallet which c...,5.0,train,1,1,0,0,1,1,"[5.0, 45.0, 55.0, 5.0, 10.0, 5.0, 35.0, 5.0, 5...",...,"[0, 1]",[maj_llama2-13b-chat_CHAT],"[0, 1, 1]","[maj_llama2-13b-chat_CHAT, maj_mistral-7b-inst...","[0, 1, 1, 0]","[maj_llama2-13b-chat_CHAT, maj_mistral-7b-inst...","[0, 1, 1, 0, 1]","[maj_llama2-13b-chat_CHAT, maj_mistral-7b-inst...","[0, 1, 1, 0, 1, 1]","[maj_llama2-13b-chat_CHAT, maj_mistral-7b-inst..."
3,"Julie is reading a 120-page book. Yesterday, s...",42.0,train,1,1,0,1,0,1,"[84.0, 42.0, 84.0, 42.0, 42.0, 42.0, 84.0, 54....",...,"[1, 0]",[maj_gemma-7b-it_CHAT],"[1, 0, 1]","[maj_gemma-7b-it_CHAT, maj_mistral-7b-inst_CHAT]","[1, 0, 1, 0]","[maj_gemma-7b-it_CHAT, maj_mistral-7b-inst_CHAT]","[1, 0, 1, 0, 1]","[maj_gemma-7b-it_CHAT, maj_mistral-7b-inst_CHA...","[1, 0, 1, 0, 1, 1]","[maj_gemma-7b-it_CHAT, maj_mistral-7b-inst_CHA..."
4,James writes a 3-page letter to 2 different fr...,624.0,train,0,1,0,0,0,0,"[312.0, 144.0, 600.0, 624.0, 468.0, 104.0, 312...",...,"[0, 0]",[],"[0, 0, 0]",[],"[0, 0, 0, 0]",[],"[0, 0, 0, 0, 0]",[],"[0, 0, 0, 0, 0, 1]",[maj_metamath-7b_LM]


In [302]:
if data_name == 'gsm8k':
    train = data_all.loc[data_all['split'] == 'train']
    test_data = data_all.loc[data_all['split'] == 'test']
    train_data, val_data = train_test_split(train, test_size=0.05, random_state=1)
    print(train_data.shape, val_data.shape, test_data.shape)
elif data_name == 'mmlu':
    train_data = data_all.loc[data_all['split'] == 'train']
    test_data = data_all.loc[data_all['split'] == 'test']
    val_data = data_all.loc[data_all['split'] == 'val']
    print(train_data.shape, val_data.shape, test_data.shape)

(6816, 25) (359, 25) (1319, 25)


In [303]:
#label Distribution 
from collections import Counter

def lab_dist(all_in):
    temp = [ [ keep_llms[i] for i, val in enumerate(item) if val !=0 ] for item in all_in]
    flattened_list = [item for sublist in temp for item in sublist]

    # Count the occurrences of each element
    element_counts = Counter(flattened_list)

    # Display the counts
    for element, count in element_counts.items():
        percentage = (count / len(all_in)) * 100
        print(f"{element}: {count} times ({percentage:.2f}%)")
        
if data_name == 'gsm8k':
    print("*********** Train Data *************")
    lab_dist(train_data['maj_6'].tolist())
    print("")
    print("*********** Validation Data ****************")
    lab_dist(val_data['maj_6'].tolist())
    print("")
    print("*********** Test Data ****************")
    lab_dist(test_data['maj_6'].tolist())
    print("*********** All Data ****************")
    lab_dist(data_all['maj_6'].tolist())
elif data_name == 'mmlu':
    print("*********** Train Data *************")
    lab_dist(train_data['maj_7'].tolist())
    print("")
    print("*********** Validation Data ****************")
    lab_dist(val_data['maj_7'].tolist())
    print("")
    print("*********** Test Data ****************")
    lab_dist(test_data['maj_7'].tolist())
    print("*********** All Data ****************")
    lab_dist(data_all['maj_7'].tolist())

*********** Train Data *************
maj_metamath-7b_LM: 6295 times (92.36%)
maj_gemma-7b_LM: 5023 times (73.69%)
maj_mistral-7b-lm_LM: 4049 times (59.40%)
maj_mistral-7b-inst_CHAT: 3830 times (56.19%)
maj_llama2-13b-chat_CHAT: 3434 times (50.38%)
maj_gemma-7b-it_CHAT: 2841 times (41.68%)

*********** Validation Data ****************
maj_metamath-7b_LM: 331 times (92.20%)
maj_gemma-7b_LM: 250 times (69.64%)
maj_llama2-13b-chat_CHAT: 180 times (50.14%)
maj_mistral-7b-lm_LM: 208 times (57.94%)
maj_mistral-7b-inst_CHAT: 182 times (50.70%)
maj_gemma-7b-it_CHAT: 145 times (40.39%)

*********** Test Data ****************
maj_metamath-7b_LM: 887 times (67.25%)
maj_gemma-7b_LM: 940 times (71.27%)
maj_mistral-7b-lm_LM: 798 times (60.50%)
maj_mistral-7b-inst_CHAT: 748 times (56.71%)
maj_llama2-13b-chat_CHAT: 649 times (49.20%)
maj_gemma-7b-it_CHAT: 557 times (42.23%)
*********** All Data ****************
maj_metamath-7b_LM: 7787 times (88.57%)
maj_gemma-7b_LM: 6430 times (73.13%)
maj_mistral-7b-

In [305]:
if data_name == 'gsm8k':
    print("Saving to GSM8K")
    train_data.to_csv('/root/COLM/dataset/gsm8k/train_all_top.csv', index=False, encoding="utf-8")
    val_data.to_csv('/root/COLM/dataset/gsm8k/val_all_top.csv', index=False, encoding="utf-8")
    data_all.to_csv('/root/COLM/dataset/gsm8k/test_all_top.csv', index=False, encoding="utf-8")
    data_all.to_csv('/root/COLM/dataset/gsm8k/gsm8k_unified_v5.csv', index=False, encoding="utf-8")
elif data_name == 'mmlu':
    print("Saving to MMLU")
    train_data.to_csv('/root/COLM/dataset/mmlu/train_all_top.csv', index=False, encoding="utf-8")
    val_data.to_csv('/root/COLM/dataset/mmlu/val_all_top.csv', index=False, encoding="utf-8")
    data_all.to_csv('/root/COLM/dataset/mmlu/test_all_top.csv', index=False, encoding="utf-8")
    data_all.to_csv('/root/COLM/dataset/mmlu/mmlu_unified_v5.csv', index=False, encoding="utf-8")